# Lifespan Age Synthesis with Geometry Guidance and Decorrelation Constraint Demo

This colab lets you try our method on your own image!

Download the github and prepare the environments

In [ ]:
!git clone https://github.com/zlq1z2l3q/GGDC
%cd GGDC/
!pip3 install -r requirements.txt

Download auxiliary models

In [ ]:
!python download_models.py

Download pretrained models.
If you cannot download the checkpoints, then you should download them locally and then upload them to the ./checkpoints/males_model and ./checkpoints/females_model

In [ ]:
!pip install --upgrade --no-cache-dir gdown

!mkdir checkpoints

%cd checkpoints

print('downloading males model')

!gdown --id '-'

!unzip males_model.zip

print('downloading females model')

!gdown --id '-'

!unzip females_model.zip

%cd ..

In [ ]:
import os
from collections import OrderedDict
from options.test_options import TestOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer
import torch

opt = TestOptions().parse(save=False)
opt.display_id = 0 # do not launch visdom
opt.nThreads = 1   # test code only supports nThreads = 1
opt.batchSize = 1  # test code only supports batchSize = 1
opt.serial_batches = True  # no shuffle
opt.no_flip = True  # no flip
opt.in_the_wild = True # This triggers preprocessing of in the wild images in the dataloader
opt.traverse = True # This tells the model to traverse the latent space between anchor classes
opt.interp_step = 0.05 # this controls the number of images to interpolate between anchor classes

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
visualizer = Visualizer(opt)

In [ ]:
opt.name = 'males_model' # change to 'females_model' if you're trying the code on a female image
model = create_model(opt)
model.eval()

In [ ]:
# upload your image (the code supports only a single image at a time)
from google.colab import files
uploaded = files.upload()
for filename in uploaded.keys():
  img_path = filename
  print('User uploaded file "{name}"'.format(name=filename))

generate output


In [ ]:
data = dataset.dataset.get_item_from_path(img_path)
visuals = model.inference(data)

os.makedirs('results', exist_ok=True)
out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.mp4')
visualizer.make_video(visuals, out_path)

You can download the video in the results folder

or

Visualize the result

In [ ]:
use_webm = False
# For some unknown reason the mp4 video is not displayed on chrome
# If you have chrome, uncomment the following lines to convert the 
# result to webm for display purposes

# !pip3 install webm
# webm_out_path = os.path.join('results', os.path.splitext(img_path)[0].replace(' ', '_') + '.webm')
# !webm -i $out_path $webm_out_path
# use_webm = True

from IPython.display import HTML
from base64 import b64encode
video_path = webm_out_path if use_webm else out_path
video_type = "video/webm" if use_webm else "video/mp4"
mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width={0} controls>
      <source src="{1}" type="{2}">
</video>
""".format(opt.fineSize, data_url, video_type))
